In [6]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [7]:
names = ["Naive Bayes", "QDA","Decision Tree", "Random Forest", "Neural Net", "AdaBoost","Nearest Neighbors", "Linear SVM", "RBF SVM"]
classifiers =[GaussianNB(),QuadraticDiscriminantAnalysis(),DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=5),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1)] 

#Pendiente , "Gaussian Process" GaussianProcessClassifier(1.0 * RBF(1.0))

In [8]:
def encode_target(df, target_column):
    """Add column to df with integers for the target.

    Args
    ----
    df -- pandas DataFrame.
    target_column -- column to map to int, producing
                     new Target column.

    Returns
    -------
    df_mod -- modified DataFrame.
    targets -- list of target names.
    """
    df_mod = df.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod["Target"] = df_mod[target_column].replace(map_to_int)

    return (df_mod, targets)

In [49]:
path = "/home/jorgher/Documents/faceRec/CFEE/"
files = sorted(glob.glob(path+'*.csv'))
files = files[:7]
emotions = ["neutral",'happy','sad','fearful','angry','surprised','disgusted']
dbs = []
for i,f in enumerate(files):
    db = pd.read_csv(f)
    db = db[['D1','D2','D3','D4','D5','D6','D7','A0','A1','A2','A3','A4','A5']]
    db['Emotions'] = emotions[i]
    dbs.append(db)

db = (pd.concat(dbs[i][:] for i in range(len(files))))
db_Train, db_Test = train_test_split(db, test_size=0.2)

In [58]:
db_7 = db_Test.sample(n=300)
test = db_7[['D1','D2','D3','D4','D5','D6','D7','A0','A1','A2','A3','A4','A5']]
df2, targets = encode_target(db_Train, "Emotions")

In [59]:
features = list(df2.columns[:13])
print("* features:", features, sep="\n")
ERtype={0:'neutral', 1: 'happy',
        2: 'sad',3: 'fearful',4:'angry',
        5: 'surprised',6: 'disgusted'}
y = df2["Target"]
X = df2[features]

* features:
['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'A0', 'A1', 'A2', 'A3', 'A4', 'A5']


In [60]:
emoT = list(db_7['Emotions'])
for name, clf in zip(names, classifiers):
        print(name)
        clf.fit(X, y)
        score = clf.score(X, y)
        print(score)
        con = 0
        for i in range(len(test)):
            e = clf.predict(test.iloc[[i]])
            if ERtype[e[0]] == emoT[i]:
                #print('Comparar: {} con: {}.'.format(ERtype[e[0]],emoT[i]))
                con = con + 1
                      
                      
        print('Porcentaje de acierto real.', con/len(test)*100)

Naive Bayes
0.616459627329
Porcentaje de acierto real. 10.333333333333334
QDA
0.729037267081
Porcentaje de acierto real. 12.666666666666668
Decision Tree
0.604037267081
Porcentaje de acierto real. 11.333333333333332
Random Forest
0.671583850932
Porcentaje de acierto real. 12.666666666666668
Neural Net
0.578416149068
Porcentaje de acierto real. 8.666666666666668
AdaBoost
0.540372670807
Porcentaje de acierto real. 7.000000000000001
Nearest Neighbors
0.730590062112
Porcentaje de acierto real. 13.333333333333334
Linear SVM
0.625776397516
Porcentaje de acierto real. 11.333333333333332
RBF SVM
1.0
Porcentaje de acierto real. 18.0
